#### _Speech Processing Labs: TTS_

<div class="alert alert-success">
<h3>Install some more python packages</h3>
This notebook needs a few more python packages further to those used in the SIGNALs notebooks. You can install them using <strong>conda</strong> in your terminal. First make sure your conda environment is active:

<pre>
<code>
conda activate slp
</code>
</pre>
    

Or replace 'slp' with the name of the conda enviroment you use with these notebooks. 
Then run this command to install the packages: 

<pre>
<code>
conda install -c conda-forge anytree urllib3 requests
</code>
</pre>

<strong>Alternatively</strong>, you could also install them using <strong>pip</strong>

<pre>
<code>
pip install anytree urllib3 requests
</code>
</pre>
You may then need to restart the kernel for this notebook: go to 'Kernel' in the menu and then 'restart and clear output'. 

</div>

In [7]:
!pip install anytree urllib3 requests


  Using cached anytree-2.8.0-py2.py3-none-any.whl (41 kB)
     |████████████████████████████████| 138 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 1.8 MB/s  eta 0:00:01
     |████████████████████████████████| 61 kB 10.7 MB/s eta 0:00:01


In [8]:
# run this first after you've installed all the imported packages (see above) 
import matplotlib.pyplot as plt
import numpy as np
import math
import re
import string
import IPython
import urllib.request
from anytree import Node, RenderTree

In [9]:
def entropy(counts):
    H=0 # entropy
    total_count=float(sum(counts))
    for c in counts:
        if c > 0: # cannot take log of zero
            p=float(c)/total_count
            H=H + p * math.log2(p)
    H=H*-1.0
    return H

# 3 Learning a complete decision tree

### Learning Outcomes
* Relate what you have learned in the previous notebooks to the full algorithm
* Understand a variety of stopping criteria

### Need to know
* The algorithm for learning a decision tree
* Topic Videos: Pronunciation, Decision tree, Learning decision trees


In the previous notebook, you ran a few iterations of the tree learning ("growing") algorithm manually. It was hard work, wasn't it! This notebook provides a complete implementation of the algorithm that runs automatically. This is a simple algorithm that matches the one described in the topic video [Learning Decision Trees](http://speech.zone/courses/speech-processing/module-4-speech-synthesis-front-end-2/videos/learning-decision-trees/). There are many possible improvements to this basic algorithm, such as more sophisticated stopping criteria, but those are beyond the scope of the course.

Try to understand the code, remembering that this is not a requirement of the course. Even if you don't understand the code, try experimenting with the stopping criteria to discover the effect on the resulting tree. Classify the test data with each tree that you build.

We're going back to the G2P task for this notebook.

## 3.1 Data

### 3.1.1 Raw data

We will use CMUdict as a source of raw data, so let's download a copy - run this code block and wait for it to print the message "loaded CMUdict". This will take a moment or two, depending on your download speed.

In [10]:
cmudict_url="http://svn.code.sf.net/p/cmusphinx/code/trunk/cmudict/cmudict.0.7a" # 4 MB download
with urllib.request.urlopen(cmudict_url) as f:
    cmudict=f.readlines()
print("loaded CMUdict")

loaded CMUdict


### 3.1.2 Feature extraction and feature engineering

Now we extract the predictors and predictee. To keep this example simple, we are only using CMUdict entries that contain a single occurrence of the letter 'c' and contain a single occurrence of one of the 4 phonemes /k/, /s/, /ʃ/, or /tʃ/. We are assuming that the letter and the phoneme correspond. As usual, we will get around the problem of words having a variable number of letters by placing a window around the letter of interest.

In reality, we would want to use all words containing one or more letter 'c', and this would involve making an alignment between letters and phonemes (e.g., using dynamic programming) to find this correspondence.

In [11]:
discard=";#(){}!?'\"%&,-.:/"
letter_of_interest="c"
phonemes_of_interest=["K","SH","S","CH"]

data=[]

for line in cmudict:
    l=line.decode() # because urllib gives us a bytestream, not text
    # discard messy entries and comments
    if all(x not in l for x in discard):
        # pad with two underscores for P PP and N NN when C is word initial or final
        g="__"+l.split("  ",1)[0].strip().lower()+"__"
        # drop lexical stress and split phonemes into a list
        p=l.split("  ",1)[1].replace("0","").replace("1","").replace("2","").strip().split()
        # only keep words with a single occurence of letter_of_interest
        if g.count(letter_of_interest) != 1: continue
        # only keep words with a single occurence of any phoneme of interest
        if sum(p.count(phoneme) for phoneme in phonemes_of_interest) !=1: continue
        which_phoneme = [phoneme for phoneme in phonemes_of_interest if(phoneme in p)][0]
        # map to the IPA because it looks prettier
        which_phoneme = which_phoneme.replace("K","k").replace("SH","ʃ").replace("S","s").replace("CH","tʃ")
        # place a window around the letter
        where=g.find(letter_of_interest)
        letters=g[where-2:where+3]
        # extract a tuple of predictors and predictee; we'll store the predictors in a string for simplicity
        data.append((letters,which_phoneme))

# we will use the IPA from now on
phonemes_of_interest=["k","s","ʃ","tʃ"]


print("Prepared the data. Here's what it looks like:")
print(data[:5],"...",data[-5:])

Prepared the data. Here's what it looks like:
[('aache', 'k'), ('aache', 'k'), ('ancor', 'k'), ('bacha', 'k'), ('back_', 'k')] ... [('zycad', 'k'), ('zych_', 'tʃ'), ('zyche', 'k'), ('deco_', 'k'), ('wicki', 'k')]


### 3.1.3 Questions

We construct a list of all possible questions that we could ask about the predictors. Each predictor value is one of the 26 letters of the alphabet, or "_".

In [12]:
# a dictionary of questions
#  key is a pretty name for a question, e.g., P=="h"
#  value is a regex that implements the question by matching the string of predictors, e.g., .h...
questions={}
# the predictors are stored in a single string, so we can write the questions as simple patterns (regular expressions)
for letter in string.ascii_lowercase+"_":
    questions["PP==\""+letter+"\""]=re.compile(letter+"....")
    questions["P==\""+letter+"\""]=re.compile("."+letter+"...")
    # questions[" C=="+letter]=re.compile(".."+letter+"..") not needed in this example since C is always the same letter
    questions["N==\""+letter+"\""]=re.compile("..."+letter+".")
    questions["NN==\""+letter+"\""]=re.compile("...."+letter)

questions    

{'PP=="a"': re.compile(r'a....', re.UNICODE),
 'P=="a"': re.compile(r'.a...', re.UNICODE),
 'N=="a"': re.compile(r'...a.', re.UNICODE),
 'NN=="a"': re.compile(r'....a', re.UNICODE),
 'PP=="b"': re.compile(r'b....', re.UNICODE),
 'P=="b"': re.compile(r'.b...', re.UNICODE),
 'N=="b"': re.compile(r'...b.', re.UNICODE),
 'NN=="b"': re.compile(r'....b', re.UNICODE),
 'PP=="c"': re.compile(r'c....', re.UNICODE),
 'P=="c"': re.compile(r'.c...', re.UNICODE),
 'N=="c"': re.compile(r'...c.', re.UNICODE),
 'NN=="c"': re.compile(r'....c', re.UNICODE),
 'PP=="d"': re.compile(r'd....', re.UNICODE),
 'P=="d"': re.compile(r'.d...', re.UNICODE),
 'N=="d"': re.compile(r'...d.', re.UNICODE),
 'NN=="d"': re.compile(r'....d', re.UNICODE),
 'PP=="e"': re.compile(r'e....', re.UNICODE),
 'P=="e"': re.compile(r'.e...', re.UNICODE),
 'N=="e"': re.compile(r'...e.', re.UNICODE),
 'NN=="e"': re.compile(r'....e', re.UNICODE),
 'PP=="f"': re.compile(r'f....', re.UNICODE),
 'P=="f"': re.compile(r'.f...', re.UNICODE),

## 3.2 The tree-growing functions

In [13]:
def grow_tree(parent_node,data,questions,predictee_values):

    best_question,entropy_reduction=find_best_question(data,questions,predictee_values)

    if best_question != None:
        yes_data,no_data = split_data(data,questions[best_question])
    else:
        yes_data=[]
        no_data=[]
        
    # compute majority class, in case this is a terminal (leaf) node
    #  (or to label non-terminal nodes out of interest)
    max_count=0
    majority_value="NONE!"
    total_count=0
    distribution=""
    predictees = [data_point[1] for data_point in data]
    for predictee_value in predictee_values:
        this_count = predictees.count(predictee_value)
        total_count = total_count + this_count
        distribution=distribution+" "+predictee_value+":"+str(this_count)
        if this_count > max_count:
            max_count = this_count
            majority_value = predictee_value

    terminate_here=False
    reason=""
    # Here are the stopping criteria - if any one of them is met, we stop
    #  you can experiment with the values in some of these
    if (best_question == None):
        reason=reason+" failed to find any question that would reduce entropy\n"
        terminate_here=True
    if (entropy_reduction < 0.2):
        reason=reason+" entropy gain would only be {:.3} bits".format(entropy_reduction)+"\n"
        terminate_here=True
    if (len(yes_data) < 200) or (len(no_data) < 200):
        reason=reason+" too few data would remain in one partition after best available split:"+str(len(yes_data))+"+"+str(len(no_data))+" data points.\n"
        terminate_here=True
    if terminate_here:
        print("TERMINATING. One or more stopping criteria were met:")
        print(reason,end='')
        print(" creating leaf node labelled with class",majority_value)
        # make this node a leaf, labelled with the majority class
        new_node = Node("/"+majority_value+"/ "+distribution, parent=parent_node)

        print("\n======== Tree currently looks like this =========")
        for pre, fill, node in RenderTree(new_node.root):
            print("%s%s" % (pre, node.name))
        print("=================================================\n")

        return

    else: 
        print("QUESTION",best_question,"will reduce entropy by {:.3} bits".format(entropy_reduction),"so adding this to the tree.")
        # label the current node with the best question
        new_node = Node(best_question+" "+str(len(data))+" /"+majority_value+"/ "+distribution, parent=parent_node)

        print("\n======== Tree currently looks like this =========")
        for pre, fill, node in RenderTree(new_node.root):
            print("%s%s" % (pre, node.name))
        print("=================================================\n")

        # recurse
        print("RECURSE from",new_node.name.split(' ')[0],"down the 'yes' branch")
        grow_tree(new_node,yes_data,questions,predictee_values)
        print("RECURSE from",new_node.name.split(' ')[0],"down the 'no' branch")
        grow_tree(new_node, no_data,questions,predictee_values)

    return new_node


def split_data(data,question,only_predictees=False):

    yes=[]
    no=[]
    for (predictors,predictee) in data:
        if question.match(predictors):
            if only_predictees:
                yes.append(predictee)
            else:
                yes.append((predictors,predictee))
        else:
            if only_predictees:
                no.append(predictee)
            else:
                no.append((predictors,predictee))
    return yes,no

def find_best_question(data,questions,predictee_values):

    parent_predictees=[predictee for (predictors,predictee) in data]
    parent_counts=[parent_predictees.count(phoneme) for phoneme in predictee_values]
    parent_entropy=entropy(parent_counts)
    best_entropy=parent_entropy
    best_question=None
    for this_question in questions:
        yes_predictees,no_predictees = split_data(data,questions[this_question],only_predictees=True)
        yes_counts=[yes_predictees.count(predictee) for predictee in predictee_values]
        no_counts =[no_predictees.count(predictee) for predictee in predictee_values]
        yes_count=sum(yes_counts)
        no_count =sum(no_counts)
        total_count = yes_count + no_count
        yes_prob=float(yes_count)/float(total_count)
        no_prob =float(no_count)/float(total_count)
        this_entropy=yes_prob*entropy(yes_counts) + no_prob*entropy(no_counts)
        if this_entropy < best_entropy:
            best_entropy = this_entropy
            best_question = this_question

    return(best_question,parent_entropy-best_entropy)

## 3.3 Run the algorithm

In [14]:
print("Starting to grow tree from "+str(len(data))+" data points using "+str(len(questions))+" questions")
root_node=grow_tree(None,data,questions,phonemes_of_interest)

Starting to grow tree from 16540 data points using 108 questions
QUESTION N=="h" will reduce entropy by 0.378 bits so adding this to the tree.

======== Tree currently looks like this =========
N=="h" 16540 /k/  k:11180 s:2185 ʃ:1170 tʃ:2005

RECURSE from N=="h" down the 'yes' branch
QUESTION P=="s" will reduce entropy by 0.484 bits so adding this to the tree.

======== Tree currently looks like this =========
N=="h" 16540 /k/  k:11180 s:2185 ʃ:1170 tʃ:2005
└── P=="s" 3737 /tʃ/  k:949 s:9 ʃ:1028 tʃ:1751

RECURSE from P=="s" down the 'yes' branch
TERMINATING. One or more stopping criteria were met:
 too few data would remain in one partition after best available split:71+830 data points.
 creating leaf node labelled with class ʃ

======== Tree currently looks like this =========
N=="h" 16540 /k/  k:11180 s:2185 ʃ:1170 tʃ:2005
└── P=="s" 3737 /tʃ/  k:949 s:9 ʃ:1028 tʃ:1751
    └── /ʃ/  k:0 s:3 ʃ:827 tʃ:71

RECURSE from P=="s" down the 'no' branch
TERMINATING. One or more stopping criteri

In the tree printed above, each non-terminal node is labelled with a question and the total number of data points that reached this node when learning the tree. See how the amount of data reduces as the tree is grown. This is a weakness of the Decision Tree as a model, because the lower parts of the tree are learned from much less data than the parts nearer the root. 

All nodes are labelled with the majority class and the distribution of predictee values.

### 3.3.1 Experiment with the stopping criteria

Try changing some of the parameters in the stopping criteria. You can make the tree smaller or larger.

## 3.4 Test

Pass the following test data through the tree and predict the phoneme. This is called "doing inference" or simply "testing".

I've provided the correct answers for the predictee so you can compute the accuracy of that prediction. Because the tree above is human-friendly you can do this without writing code. In the tree, the "yes" branch is printed first, then the "no branch".

(But, if you want some coding practice, then implement it in Python.)

In [15]:
test_data=[('ieced','s'),('_scho','ʃ'),('__cal','k'),('gic__','k'),('arcos','k'),('__che','tʃ'),('orca_','k'),('duca_','k'),('__cir','s'),('__cha','tʃ'),('__car','k'),('recor','k'),('focht','k'),('__cha','tʃ'),('__cha','tʃ'),('ercei','s'),('racto','k'),('uechn','k'),('recia','tʃ'),('decli','k'),('licat','k'),('rich_','k'),('pecor','k'),('dacit','s'),('rick_','k'),('decad','k'),('__cot','k'),('__cha','tʃ'),('__cen','s'),('mac__','k'),('recei','s'),('nic__','k'),('_acqu','k'),('anca_','k'),('ouch_','tʃ'),('__car','k'),('isch_','ʃ'),('__cla','k'),('__chi','tʃ'),('_mcle','k')]

Write your answers for the test data here...